In [7]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torchvision.models.detection
from torchvision.models.detection import FasterRCNN_ResNet50_FPN_Weights
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torchvision.ops import box_iou
from sklearn.metrics import average_precision_score
from collections import defaultdict


# Đường dẫn dataset
DATASET_DIR = 'datasets'
IMAGES_DIR = os.path.join(DATASET_DIR, 'ExDark')
ANNOTATIONS_DIR = os.path.join(DATASET_DIR, 'ExDark_Annno') # Hàm đọc annotations từ cấu trúc thư mục phân cấp

In [8]:
# Hàm đọc annotations từ cấu trúc thư mục phân cấp
def read_annotations(annotations_dir, images_dir):
    annotations = {}
    
    # Duyệt qua tất cả các thư mục class trong ExDark_Annno
    if not os.path.exists(annotations_dir):
        print(f"Annotations directory not found: {annotations_dir}")
        return {}
    
    for class_folder in os.listdir(annotations_dir):
        class_anno_path = os.path.join(annotations_dir, class_folder)
        if not os.path.isdir(class_anno_path):
            continue
            
        # Duyệt qua thư mục con (có thể có thêm 1 lớp thư mục class)
        for subfolder in os.listdir(class_anno_path):
            subfolder_path = os.path.join(class_anno_path, subfolder)
            if os.path.isdir(subfolder_path):
                # Nếu có thêm 1 lớp thư mục con
                annotation_files_path = subfolder_path
                # print(f"Found subfolder for class {class_folder}: {subfolder_path}")
            else:
                # Nếu file annotation nằm trực tiếp trong thư mục class
                annotation_files_path = class_anno_path
                # print(f"Using class folder for annotations: {class_anno_path}")
                break
        
        # Đọc các file annotation
        if os.path.exists(annotation_files_path):
            for filename in os.listdir(annotation_files_path):
                if filename.endswith('.txt'):
                    # Tìm file ảnh tương ứng
                    img_name_base = filename.replace('.txt', '')
                    
                    # Tìm file ảnh trong thư mục class tương ứng
                    img_class_path = os.path.join(images_dir, class_folder)
                    img_path = None
                    
                    if os.path.exists(img_class_path):
                        for img_file in os.listdir(img_class_path):
                            if img_file.startswith(img_name_base):
                                img_path = os.path.join(class_folder, img_file)
                                # print(f"Found image for annotation {filename}: {img_path}")
                                break
                    
                    if img_path is None:
                        continue
                    
                    # Đọc annotations từ file txt
                    anno_file_path = os.path.join(annotation_files_path, filename)
                    with open(anno_file_path, 'r') as f:
                        lines = f.readlines()
                        objs = []
                        
                        for line in lines:
                            line = line.strip()
                            if line.startswith('%') or not line:  # Bỏ qua comment và dòng trống
                                continue
                                
                            parts = line.split()
                            if len(parts) < 7:  # Ít nhất cần có label, width, height, xmin, ymin, xmax, ymax
                                continue
                            
                            try:
                                label = parts[0]
                                xmin = float(parts[1])
                                ymin = float(parts[2])
                                bbox_width = float(parts[3])
                                bbox_height = float(parts[4])
                                
                                # Chuyển đổi sang format [x
                                xmax = xmin + bbox_width
                                ymax = ymin + bbox_height
                                bbox = [xmin, ymin, xmax, ymax]
                                
                                objs.append({
                                    'label': label,
                                    'bbox': bbox
                                })
                                
                                # print(f"Parsed object: {label}, bbox: {bbox}, img_size: ({width}, {height})")
                            except ValueError:
                                print(f"Error parsing line in {filename}: {line}")
                                continue
                        
                        if objs:  # Chỉ thêm vào nếu có objects
                            annotations[img_path] = objs
    
    print(f"Loaded annotations for {len(annotations)} images")
    return annotations


In [9]:
# Hàm tạo label map từ annotations
def create_label_map(annotations):
    label2idx = {}
    idx = 0
    for objs in annotations.values():
        for obj in objs:
            label = obj['label']
            if label not in label2idx:
                label2idx[label] = idx
                idx += 1
    return label2idx

In [10]:
# Dataset cho object detection
class ExDarkDataset(Dataset):
    def __init__(self, img_dir, annotations, label2idx, transform=None):
        self.img_dir = img_dir
        self.annotations = annotations
        self.label2idx = label2idx
        self.transform = transform
        self.img_files = list(annotations.keys())
        
        print(f"Dataset initialized with {len(self.img_files)} images")
        if len(self.img_files) > 0:
            print(f"Sample image paths: {self.img_files[:10]}")

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_file = self.img_files[idx]
        img_full_path = os.path.join(self.img_dir, img_file)
        
        # Kiểm tra file có tồn tại không
        if not os.path.exists(img_full_path):
            print(f"Image not found: {img_full_path}")
            # Thử tìm với các extension khác
            base_path = os.path.splitext(img_full_path)[0]
            for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
                if os.path.exists(base_path + ext):
                    img_full_path = base_path + ext
                    break
        
        try:
            image = Image.open(img_full_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_full_path}: {e}")
            # Tạo ảnh trắng thay thế
            image = Image.new('RGB', (224, 224), color='white')

        objs = self.annotations[img_file]
        boxes = []
        labels = []
        
        for obj in objs:
            bbox = obj['bbox']
            # Đảm bảo bbox có định dạng [xmin, ymin, xmax, ymax] cho Faster R-CNN
            xmin, ymin, xmax, ymax = bbox
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(self.label2idx[obj['label']])

        # Chuyển đổi sang tensor
        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)
        
        # Tạo target dict
        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])

        if self.transform:
            image = self.transform(image)

        return image, target

# Transform ảnh
train_transform = transforms.Compose([
    transforms.ToTensor()
])

# Đọc annotations và tạo label map
print("Loading annotations...")
annotations = read_annotations(ANNOTATIONS_DIR, IMAGES_DIR)

print("Creating label map...")
label2idx = create_label_map(annotations)

Loading annotations...
Loaded annotations for 7361 images
Creating label map...


In [11]:
# Tạo dataset
print("Creating dataset...")
dataset = ExDarkDataset(
    img_dir=IMAGES_DIR,
    annotations=annotations,
    label2idx=label2idx,
    transform=train_transform
)
print(f"Dataset size: {len(dataset)}")
try:
    sample_img, sample_target = dataset[0]
    print(f"Sample image shape: {sample_img.shape}")
    print(f"Sample target: {sample_target}")
except Exception as e:
    print(f"Error loading sample: {e}")

# Chia dataset train/val/test
def split_dataset(dataset, train_ratio=0.7, val_ratio=0.1):
    total = len(dataset)
    train_len = int(total * train_ratio)
    val_len = int(total * val_ratio)
    test_len = total - train_len - val_len
    return random_split(dataset, [train_len, val_len, test_len])

train_set, val_set, test_set = split_dataset(dataset)
print(f"Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")

# Custom collate function cho object detection
def collate_fn(batch):
    return tuple(zip(*batch))

train_loader = DataLoader(train_set, batch_size=4, shuffle=True, collate_fn=collate_fn)
val_loader = DataLoader(val_set, batch_size=4, shuffle=False, collate_fn=collate_fn)
test_loader = DataLoader(test_set, batch_size=4, shuffle=False, collate_fn=collate_fn)

Creating dataset...
Dataset initialized with 7361 images
Sample image paths: ['Bicycle\\2015_00001.png', 'Bicycle\\2015_00002.png', 'Bicycle\\2015_00003.png', 'Bicycle\\2015_00004.jpg', 'Bicycle\\2015_00005.jpg', 'Bicycle\\2015_00006.jpg', 'Bicycle\\2015_00007.jpg', 'Bicycle\\2015_00008.jpg', 'Bicycle\\2015_00009.jpg', 'Bicycle\\2015_00010.jpg']
Dataset size: 7361
Sample image shape: torch.Size([3, 375, 500])
Sample target: {'boxes': tensor([[204.,  28., 475., 221.]]), 'labels': tensor([0]), 'image_id': tensor([0])}
Train: 5152, Val: 736, Test: 1473


In [ ]:
def get_edge_boxes(image, max_boxes=100, min_score=0.01):
    edge_boxes = cv2.ximgproc.createEdgeBoxes()
    edge_boxes.setMaxBoxes(max_boxes)
    edge_boxes.setMinScore(min_score)

    # Convert to numpy array BGR
    image_np = np.array(image)
    if image_np.ndim == 2:
        image_np = cv2.cvtColor(image_np, cv2.COLOR_GRAY2BGR)
    else:
        image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    boxes, scores = edge_boxes.getBoundingBoxes(image_np)
    return boxes, scores

def evaluate_edge_boxes(dataset, iou_threshold=0.5, max_boxes=100):
    all_ious = []
    ap_scores = []

    for img, target in tqdm(dataset, desc="Evaluating EdgeBoxes"):
        boxes_pred, _ = get_edge_boxes(transforms.ToPILImage()(img), max_boxes=max_boxes)
        boxes_pred = torch.tensor(boxes_pred[:, [0, 1, 0, 1]] + boxes_pred[:, [2, 3, 2, 3]], dtype=torch.float32) / 2
        boxes_pred[:, 2] = boxes_pred[:, 0] + (boxes_pred[:, 2] - boxes_pred[:, 0])
        boxes_pred[:, 3] = boxes_pred[:, 1] + (boxes_pred[:, 3] - boxes_pred[:, 1])

        gt_boxes = target['boxes']

        if len(boxes_pred) == 0 or len(gt_boxes) == 0:
            continue

        ious = box_iou(gt_boxes, boxes_pred)  # [num_gt, num_pred]
        max_ious, _ = torch.max(ious, dim=1)  # best proposal for each GT box

        correct = (max_ious >= iou_threshold).float()
        precision = correct.sum() / len(boxes_pred)
        recall = correct.sum() / len(gt_boxes)
        ap_scores.append(average_precision_score(correct.numpy(), max_ious.numpy()))

        all_ious.extend(max_ious.tolist())

    mean_ap = np.mean(ap_scores) if ap_scores else 0.0
    print(f"\nMean AP @ IOU={iou_threshold:.2f}: {mean_ap:.4f}")
    return mean_ap, all_ious

# Đánh giá Edge Boxes trên tập train
print("Evaluating Edge Boxes on train set...")
mean_ap_train, ious_train = evaluate_edge_boxes(train_loader.dataset)
# Đánh giá Edge Boxes trên tập val
print("Evaluating Edge Boxes on validation set...")
mean_ap_val, ious_val = evaluate_edge_boxes(val_loader.dataset)
# Đánh giá Edge Boxes trên tập test
print("Evaluating Edge Boxes on test set...")
mean_ap_test, ious_test = evaluate_edge_boxes(test_loader.dataset)

Evaluating Edge Boxes on train set...


Evaluating EdgeBoxes:   0%|          | 0/5152 [00:00<?, ?it/s]d:\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
Evaluating EdgeBoxes:   0%|          | 1/5152 [00:00<13:54,  6.17it/s]d:\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
d:\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
Evaluating EdgeBoxes:   0%|          | 3/5152 [00:00<07:02, 12.19it/s]d:\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall is set to one for all thresholds.
  warnings.warn(
d:\Anaconda\Lib\site-packages\sklearn\metrics\_ranking.py:1030: UserWarning: No positive class found in y_true, recall

TypeError: tuple indices must be integers or slices, not tuple

In [15]:
from PIL import ImageDraw

def visualize_proposals(img_tensor, boxes, top_n=10):
    img = transforms.ToPILImage()(img_tensor).copy()
    draw = ImageDraw.Draw(img)

    for i in range(min(top_n, len(boxes))):
        x, y, w, h = boxes[i]
        draw.rectangle([x, y, x + w, y + h], outline='red', width=2)
    return img

# Ví dụ dùng thử:
img, target = dataset[0]
boxes, _ = get_edge_boxes(transforms.ToPILImage()(img))
vis_img = visualize_proposals(img, boxes)
vis_img.show()


In [ ]:
import os
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from tqdm import tqdm
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import average_precision_score
from torchvision.ops import box_iou

os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# Đường dẫn dataset
DATASET_DIR = 'datasets'
IMAGES_DIR = os.path.join(DATASET_DIR, 'ExDark')
ANNOTATIONS_DIR = os.path.join(DATASET_DIR, 'ExDark_Annno')

# Hàm đọc annotations từ cấu trúc thư mục phân cấp
def read_annotations(annotations_dir, images_dir):
    annotations = {}
    
    if not os.path.exists(annotations_dir):
        print(f"Annotations directory not found: {annotations_dir}")
        return {}

    for class_folder in os.listdir(annotations_dir):
        class_anno_path = os.path.join(annotations_dir, class_folder)
        if not os.path.isdir(class_anno_path):
            continue

        for subfolder in os.listdir(class_anno_path):
            subfolder_path = os.path.join(class_anno_path, subfolder)
            if os.path.isdir(subfolder_path):
                annotation_files_path = subfolder_path
            else:
                annotation_files_path = class_anno_path
                break

        if os.path.exists(annotation_files_path):
            for filename in os.listdir(annotation_files_path):
                if filename.endswith('.txt'):
                    img_name_base = filename.replace('.txt', '')
                    img_class_path = os.path.join(images_dir, class_folder)
                    img_path = None

                    if os.path.exists(img_class_path):
                        for img_file in os.listdir(img_class_path):
                            if img_file.startswith(img_name_base):
                                img_path = os.path.join(class_folder, img_file)
                                break

                    if img_path is None:
                        continue

                    anno_file_path = os.path.join(annotation_files_path, filename)
                    with open(anno_file_path, 'r') as f:
                        lines = f.readlines()
                    objs = []

                    for line in lines:
                        line = line.strip()
                        if line.startswith('%') or not line:
                            continue

                        parts = line.split()
                        if len(parts) < 5:
                            continue

                        try:
                            label = parts[0]
                            xmin = float(parts[1])
                            ymin = float(parts[2])
                            bbox_width = float(parts[3])
                            bbox_height = float(parts[4])

                            xmax = xmin + bbox_width
                            ymax = ymin + bbox_height
                            bbox = [xmin, ymin, xmax, ymax]

                            objs.append({
                                'label': label,
                                'bbox': bbox
                            })
                        except ValueError:
                            print(f"Error parsing line in {filename}: {line}")
                            continue

                    if objs:
                        annotations[img_path] = objs

    print(f"Loaded annotations for {len(annotations)} images")
    return annotations

def create_label_map(annotations):
    label2idx = {}
    idx = 1
    for objs in annotations.values():
        for obj in objs:
            label = obj['label']
            if label not in label2idx:
                label2idx[label] = idx
                idx += 1
    return label2idx

class ExDarkDataset(Dataset):
    def __init__(self, img_dir, annotations, label2idx, transform=None):
        self.img_dir = img_dir
        self.annotations = annotations
        self.label2idx = label2idx
        self.transform = transform
        self.img_files = list(annotations.keys())
        print(f"Dataset initialized with {len(self.img_files)} images")

    def __len__(self):
        return len(self.img_files)

    def __getitem__(self, idx):
        img_file = self.img_files[idx]
        img_full_path = os.path.join(self.img_dir, img_file)

        if not os.path.exists(img_full_path):
            base_path = os.path.splitext(img_full_path)[0]
            for ext in ['.jpg', '.png', '.jpeg', '.JPG']:
                if os.path.exists(base_path + ext):
                    img_full_path = base_path + ext
                    break

        try:
            image = Image.open(img_full_path).convert("RGB")
        except Exception as e:
            print(f"Error loading image {img_full_path}: {e}")
            image = Image.new('RGB', (224, 224), color='white')

        objs = self.annotations[img_file]
        boxes = []
        labels = []

        for obj in objs:
            bbox = obj['bbox']
            xmin, ymin, xmax, ymax = bbox
            boxes.append([xmin, ymin, xmax, ymax])
            labels.append(self.label2idx[obj['label']])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target = {}
        target['boxes'] = boxes
        target['labels'] = labels
        target['image_id'] = torch.tensor([idx])

        if self.transform:
            image = self.transform(image)

        return image, target

# Enhanced EdgeBoxes implementation for low-light conditions
def preprocess_low_light_image(image):
    """
    Preprocess low-light images to enhance edge detection
    """
    # Convert PIL to numpy array
    if isinstance(image, Image.Image):
        img_np = np.array(image)
    else:
        img_np = image
    
    # Convert RGB to BGR for OpenCV
    if len(img_np.shape) == 3:
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
    else:
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_GRAY2BGR)
    
    # Apply CLAHE (Contrast Limited Adaptive Histogram Equalization)
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    lab[:,:,0] = clahe.apply(lab[:,:,0])
    enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    
    # Gamma correction for low-light enhancement
    gamma = 1.5
    enhanced = np.power(enhanced / 255.0, 1.0 / gamma)
    enhanced = (enhanced * 255).astype(np.uint8)
    
    # Bilateral filter to reduce noise while preserving edges
    enhanced = cv2.bilateralFilter(enhanced, 9, 75, 75)
    
    return enhanced

def get_edge_boxes(image, max_boxes=100, min_score=0.01, enhance_low_light=True):
    """
    Extract object proposals using EdgeBoxes algorithm
    Enhanced for low-light conditions
    """
    try:
        # Preprocess for low-light conditions
        if enhance_low_light:
            image_processed = preprocess_low_light_image(image)
        else:
            if isinstance(image, Image.Image):
                image_processed = np.array(image)
                if len(image_processed.shape) == 3:
                    image_processed = cv2.cvtColor(image_processed, cv2.COLOR_RGB2BGR)
                else:
                    image_processed = cv2.cvtColor(image_processed, cv2.COLOR_GRAY2BGR)
        
        # Create EdgeBoxes detector
        edge_detection = cv2.ximgproc.createStructuredEdgeDetection('model.yml')  # You need to download this model
        
        # Alternative: Use Canny edge detection if structured edge model is not available
        gray = cv2.cvtColor(image_processed, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray, 50, 150, apertureSize=3)
        edges = edges.astype(np.float32) / 255.0
        
        # Create EdgeBoxes generator
        edge_boxes = cv2.ximgproc.createEdgeBoxes()
        edge_boxes.setMaxBoxes(max_boxes)
        edge_boxes.setMinScore(min_score)
        edge_boxes.setAlpha(0.65)
        edge_boxes.setBeta(0.75)
        edge_boxes.setEta(1.0)
        edge_boxes.setMinBoxArea(1000)
        edge_boxes.setMaxAspectRatio(3.0)
        edge_boxes.setClusterMinMag(0.5)
        
        # Get bounding boxes
        boxes, scores = edge_boxes.getBoundingBoxes(edges, image_processed)
        
        return boxes, scores
        
    except Exception as e:
        print(f"Error in get_edge_boxes: {e}")
        # Fallback: return empty boxes
        return np.array([]), np.array([])

def convert_boxes_format(boxes):
    """
    Convert boxes from [x, y, w, h] to [x1, y1, x2, y2] format
    """
    if len(boxes) == 0:
        return torch.tensor([], dtype=torch.float32).reshape(0, 4)
    
    # boxes are in format [x, y, w, h]
    x1 = boxes[:, 0]
    y1 = boxes[:, 1]
    x2 = boxes[:, 0] + boxes[:, 2]
    y2 = boxes[:, 1] + boxes[:, 3]
    
    converted_boxes = np.column_stack([x1, y1, x2, y2])
    return torch.tensor(converted_boxes, dtype=torch.float32)

def evaluate_edge_boxes(dataset, iou_threshold=0.5, max_boxes=100):
    """
    Evaluate EdgeBoxes performance on the dataset
    """
    all_ious = []
    precision_scores = []
    recall_scores = []
    
    for i in tqdm(range(len(dataset)), desc="Evaluating EdgeBoxes"):
        try:
            img, target = dataset[i]
            
            # Convert tensor to PIL Image for EdgeBoxes
            if isinstance(img, torch.Tensor):
                img_pil = transforms.ToPILImage()(img)
            else:
                img_pil = img
            
            # Get EdgeBoxes proposals
            boxes_pred, scores = get_edge_boxes(img_pil, max_boxes=max_boxes)
            
            if len(boxes_pred) == 0:
                continue
            
            # Convert to proper format [x1, y1, x2, y2]
            boxes_pred_tensor = convert_boxes_format(boxes_pred)
            gt_boxes = target['boxes']
            
            if len(boxes_pred_tensor) == 0 or len(gt_boxes) == 0:
                continue
            
            # Calculate IoU between predictions and ground truth
            ious = box_iou(gt_boxes, boxes_pred_tensor)  # [num_gt, num_pred]
            
            if ious.numel() == 0:
                continue
            
            # Find best matching proposal for each ground truth box
            max_ious, best_pred_idx = torch.max(ious, dim=1)
            
            # Calculate metrics
            tp = (max_ious >= iou_threshold).sum().item()
            fp = len(boxes_pred_tensor) - tp
            fn = len(gt_boxes) - tp
            
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
            
            precision_scores.append(precision)
            recall_scores.append(recall)
            all_ious.extend(max_ious.tolist())
            
        except Exception as e:
            print(f"Error processing sample {i}: {e}")
            continue
    
    # Calculate overall metrics
    mean_precision = np.mean(precision_scores) if precision_scores else 0.0
    mean_recall = np.mean(recall_scores) if recall_scores else 0.0
    mean_iou = np.mean(all_ious) if all_ious else 0.0
    
    print(f"\nEdgeBoxes Evaluation Results:")
    print(f"Mean Precision @ IoU={iou_threshold:.2f}: {mean_precision:.4f}")
    print(f"Mean Recall @ IoU={iou_threshold:.2f}: {mean_recall:.4f}")
    print(f"Mean IoU: {mean_iou:.4f}")
    print(f"F1-Score: {2 * mean_precision * mean_recall / (mean_precision + mean_recall) if (mean_precision + mean_recall) > 0 else 0.0:.4f}")
    
    return mean_precision, mean_recall, all_ious

# Transform ảnh
train_transform = transforms.Compose([
    transforms.ToTensor()
])

# Main execution
if __name__ == "__main__":
    # Load annotations and create label map
    print("Loading annotations...")
    annotations = read_annotations(ANNOTATIONS_DIR, IMAGES_DIR)
    
    print("Creating label map...")
    label2idx = create_label_map(annotations)
    print(f"Label map: {label2idx}")
    
    # Create dataset
    print("Creating dataset...")
    dataset = ExDarkDataset(
        img_dir=IMAGES_DIR,
        annotations=annotations,
        label2idx=label2idx,
        transform=train_transform
    )
    print(f"Dataset size: {len(dataset)}")
    
    # Test sample loading
    try:
        sample_img, sample_target = dataset[0]
        print(f"Sample image shape: {sample_img.shape}")
        print(f"Sample target: {sample_target}")
    except Exception as e:
        print(f"Error loading sample: {e}")
    
    # Split dataset
    def split_dataset(dataset, train_ratio=0.7, val_ratio=0.1):
        total = len(dataset)
        train_len = int(total * train_ratio)
        val_len = int(total * val_ratio)
        test_len = total - train_len - val_len
        return random_split(dataset, [train_len, val_len, test_len])
    
    train_set, val_set, test_set = split_dataset(dataset)
    print(f"Train: {len(train_set)}, Val: {len(val_set)}, Test: {len(test_set)}")
    
    # Custom collate function
    def collate_fn(batch):
        return tuple(zip(*batch))
    
    train_loader = DataLoader(train_set, batch_size=4, shuffle=True, collate_fn=collate_fn)
    val_loader = DataLoader(val_set, batch_size=4, shuffle=False, collate_fn=collate_fn)
    test_loader = DataLoader(test_set, batch_size=4, shuffle=False, collate_fn=collate_fn)
    
    # Evaluate EdgeBoxes on different sets
    print("\nEvaluating EdgeBoxes on train set...")
    mean_precision_train, mean_recall_train, ious_train = evaluate_edge_boxes(train_set)
    
    print("\nEvaluating EdgeBoxes on validation set...")
    mean_precision_val, mean_recall_val, ious_val = evaluate_edge_boxes(val_set)
    
    print("\nEvaluating EdgeBoxes on test set...")
    mean_precision_test, mean_recall_test, ious_test = evaluate_edge_boxes(test_set)
    
    # Plot results
    plt.figure(figsize=(12, 4))
    
    # Precision comparison
    plt.subplot(1, 3, 1)
    sets = ['Train', 'Val', 'Test']
    precisions = [mean_precision_train, mean_precision_val, mean_precision_test]
    plt.bar(sets, precisions)
    plt.title('Mean Precision by Dataset Split')
    plt.ylabel('Precision')
    
    # Recall comparison
    plt.subplot(1, 3, 2)
    recalls = [mean_recall_train, mean_recall_val, mean_recall_test]
    plt.bar(sets, recalls)
    plt.title('Mean Recall by Dataset Split')
    plt.ylabel('Recall')
    
    # IoU distribution
    plt.subplot(1, 3, 3)
    if ious_val:
        plt.hist(ious_val, bins=20, alpha=0.7)
        plt.title('IoU Distribution (Validation Set)')
        plt.xlabel('IoU')
        plt.ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()

Loading annotations...
Loaded annotations for 7361 images
Creating label map...
Label map: {'Bicycle': 1, 'Car': 2, 'Bus': 3, 'Chair': 4, 'People': 5, 'Motorbike': 6, 'Dog': 7, 'Table': 8, 'Bottle': 9, 'Boat': 10, 'Cat': 11, 'Cup': 12}
Creating dataset...
Dataset initialized with 7361 images
Dataset size: 7361
Sample image shape: torch.Size([3, 375, 500])
Sample target: {'boxes': tensor([[204.,  28., 475., 221.]]), 'labels': tensor([1]), 'image_id': tensor([0])}
Train: 5152, Val: 736, Test: 1473

Evaluating EdgeBoxes on train set...


Evaluating EdgeBoxes:   0%|          | 1/5152 [00:00<11:51,  7.24it/s]

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'



Evaluating EdgeBoxes:   0%|          | 3/5152 [00:00<07:10, 11.97it/s]

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-2

...

Evaluating EdgeBoxes:  66%|██████▌   | 3390/5152 [07:23<01:43, 17.10it/s]

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'



Evaluating EdgeBoxes:  66%|██████▌   | 3392/5152 [07:24<01:52, 15.60it/s]

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-215:Assertion failed) modelFile.isOpened() in function 'cv::ximgproc::StructuredEdgeDetectionImpl::StructuredEdgeDetectionImpl'

Error in get_edge_boxes: OpenCV(4.11.0) D:\a\opencv-python\opencv-python\opencv_contrib\modules\ximgproc\src\structured_edge_detection.cpp:441: error: (-2

Evaluating EdgeBoxes:  66%|██████▌   | 3394/5152 [07:24<01:53, 15.46it/s]